# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
F. Walter  ->  F. Walter  |  ['F. Walter']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
S. Bosman  ->  S. Bosman  |  ['S. Bosman']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
C. Bailer-Jones  ->  C. Bailer-Jones  |  ['C. Bailer-Jones']
M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']
T. Müller  ->  T. Müller  |  ['T. Müller']


T. Ueda  ->  T. Ueda  |  ['T. Ueda']
M. Flock  ->  M. Flock  |  ['M. Flock']
P. Sudarshan  ->  P. Sudarshan  |  ['P. Sudarshan']
Arxiv has 67 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2406.06674


extracting tarball to tmp_2406.06674... done.
Retrieving document from  https://arxiv.org/e-print/2406.06697


extracting tarball to tmp_2406.06697...

 done.


...

 done.
Retrieving document from  https://arxiv.org/e-print/2406.07427
extracting tarball to tmp_2406.07427...

/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2119/1211882699.py:51: LatexWarning: 2406.06740 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


 done.


list index out of range
Retrieving document from  https://arxiv.org/e-print/2406.07439
extracting tarball to tmp_2406.07439... done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2119/1211882699.py:51: LatexWarning: 2406.07439 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.06697-b31b1b.svg)](https://arxiv.org/abs/2406.06697) | **A quasar-galaxy merger at $z\sim 6.2$: rapid host growth via accretion of two massive satellite galaxies**  |
|| R. Decarli, et al. -- incl., <mark>F. Walter</mark>, <mark>E. Bañados</mark>, <mark>S. Bosman</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-06-12*|
|*Comments*| *15 pages, 16 figures. Accepted for publication in A&A*|
|**Abstract**|            We present JWST/NIRSpec Integral Field Spectroscopy in the rest-frame optical bands of the system PJ308-21, a quasar at $z=6.2342$ caught as its host galaxy interacts with companion galaxies. We detect spatially extended emission of several emission lines (H$\alpha$, H$\beta$, [OIII], [NII], [SII], HeII), which we use to study the properties of the ionized phase of the interstellar medium: the source and hardness of the photoionizing radiation field, metallicity, dust reddening, electron density and temperature, and star formation. We also marginally detect continuum starlight emission associated with the companion sources. We find that at least two independent satellite galaxies are part of the system. While the quasar host appears highly enriched and obscured, with AGN-like photoionization conditions, the western companion shows minimal dust extinction, low metallicity ($Z\sim0.4$ Z$_\odot$), and star-formation driven photoionization. The eastern companion shows higher extinction and metallicity ($Z\sim0.8$ Z$_\odot$) compared to the western companion, and it is at least partially photoionized by the nearby quasar. We do not find any indication of AGN in the companion sources. Our study shows that while the quasar host galaxy is already very massive ($M_{\rm dyn}>10^{11}$ M$_\odot$), it is still rapidly building up by accreting two relatively massive ($M_{\rm star}\sim 10^{10}$ M$_\odot$) companion sources. This dataset showcases the power of JWST in exposing the build-up of massive galaxies in the first Gyr of the Universe.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.07427-b31b1b.svg)](https://arxiv.org/abs/2406.07427) | **Support for fragile porous dust in a gravitationally self-regulated disk around IM Lup**  |
|| <mark>T. Ueda</mark>, et al. -- incl., <mark>M. Flock</mark>, <mark>P. Sudarshan</mark> |
|*Appeared on*| *2024-06-12*|
|*Comments*| *Accepted for publication. Author version*|
|**Abstract**|            Protoplanetary disks, the birthplace of planets, are expected to be gravitationally unstable in their early phase of evolution. IM Lup, a well-known T-Tauri star, is surrounded by a protoplanetary disk with spiral arms likely caused by gravitational instability. The IM Lup disk has been observed using various methods, but developing a unified explanatory model is challenging. Here we present a physical model of the IM Lup disk that offers a comprehensive explanation for diverse observations spanning from near-infrared to millimeter wavelengths. Our findings underscore the importance of dust fragility in retaining the observed millimeter emission and reveal the preference for moderately porous dust to explain observed millimeter polarization. We also find that the inner disk region is likely heated by gas accretion, providing a natural explanation for bright millimeter emission within 20 au. The actively heated inner region in the model casts a 100-au-scale shadow, aligning seamlessly with the near-infrared scattered light observation. The presence of accretion heating also supports the fragile dust scenario in which accretion efficiently heat the disk midplane. Due to the fragility of dust, it is unlikely that a potential embedded planet at 100 au formed via pebble accretion in a smooth disk, pointing to local dust enhancement boosting pebble accretion or alternative pathways such as outward migration or gravitational fragmentation.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.06674-b31b1b.svg)](https://arxiv.org/abs/2406.06674) | **Systematic Collapse of the Accretion Disc Across the Supermassive Black Hole Population**  |
|| S. Hagen, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-06-12*|
|*Comments*| *10 pages, 5 figure, 1 appendix - submitted to MNRAS, comments welcome*|
|**Abstract**|            The structure of the accretion flow onto supermassive black holes (SMBH) is not well understood. Standard disc models match to zeroth order in predicting substantial energy dissipation within optically-thick material producing a characteristic strong blue/UV continuum. However they fail at reproducing more detailed comparisons to the observed spectral shapes along with their observed variability. Based on stellar mass black holes within our galaxy, accretion discs should undergo a transition into an X-ray hot, radiatively inefficient flow, below a (mass scaled) luminosity of $\sim 0.02\,L_{\rm{Edd}}$. While this has been seen in limited samples of nearby low-luminosity active galactic nuclei (AGN) and a few rare changing-look AGN, it is not at all clear whether this transition is present in the wider AGN population across cosmic time. A key issue is the difficulty in disentangling a change in spectral state from increased dust obscuration and/or host galaxy contamination, effectively drowning out the AGN emission. Here we use the new eROSITA eFEDS Survey to identify unobscured AGN from their X-ray emission, matched to excellent optical imaging from Subaru's Hyper Suprime-Cam; allowing the subtraction of the host galaxy contamination. The resulting, uncontaminated, AGN spectra reveal a smooth transition from a strongly disc dominated state in bright AGN, to the collapse of the disc into an inefficient X-ray plasma in the low luminosity AGN, with the transition occurring at $\sim 0.02\,L_{\rm{Edd}}$; revealing fundamental aspects of accretion physics in AGN.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.06740-b31b1b.svg)](https://arxiv.org/abs/2406.06740) | **All-sky three-dimensional dust density and extinction Maps of the Milky Way out to 2.8 kpc**  |
|| T. E. Dharmawardena, et al. -- incl., <mark>C. Bailer-Jones</mark>, <mark>M. Fouesneau</mark>, <mark>T. Müller</mark> |
|*Appeared on*| *2024-06-12*|
|*Comments*| *Accepted for publication in MNRAS; 13 pages in main document and 7 pages in appendix*|
|**Abstract**|            Three-dimensional dust density maps are crucial for understanding the structure of the interstellar medium of the Milky Way and the processes that shape it. However, constructing these maps requires large datasets and the methods used to analyse them are computationally expensive and difficult to scale up. As a result it is has only recently become possible to map kiloparsec-scale regions of our Galaxy at parsec-scale grid sampling. We present all-sky three-dimensional dust density and extinction maps of the Milky Way out to 2.8~kpc in distance from the Sun using the fast and scalable Gaussian Process algorithm \DustT. The sampling of the three-dimensional map is $l,b,d = 1^{\circ} \times1^{\circ} \times 1.7$~pc. The input extinction and distance catalogue contains 120 million stars with photometry and astrometry from Gaia DR2, 2MASS and AllWISE. This combines the strengths of optical and infrared data to probe deeper into the dusty regions of the Milky Way. We compare our maps with other published 3D dust maps. All maps quantitatively agree at the $0.001$~mag~pc$^{-1}$ scale with many qualitatively similar features, although each map also has its own features. We recover Galactic features previously identified in the literature. Moreover, we also see a large under-density that may correspond to an inter-arm or -spur gap towards the Galactic Centre.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2406.07439-b31b1b.svg)](https://arxiv.org/abs/2406.07439) | **Search for photons above 10$^{18}$ eV by simultaneously measuring the atmospheric depth and the muon content of air showers at the Pierre Auger Observatory**  |
|| P. A. Collaboration, et al. |
|*Appeared on*| *2024-06-12*|
|*Comments*| *19 pages, 22 figures*|
|**Abstract**|            The Pierre Auger Observatory is the most sensitive instrument to detect photons with energies above $10^{17}$ eV. It measures extensive air showers generated by ultra high energy cosmic rays using a hybrid technique that exploits the combination of a fluorescence detector with a ground array of particle detectors. The signatures of a photon-induced air shower are a larger atmospheric depth of the shower maximum ($X_{max}$) and a steeper lateral distribution function, along with a lower number of muons with respect to the bulk of hadron-induced cascades. In this work, a new analysis technique in the energy interval between 1 and 30 EeV (1 EeV = $10^{18}$ eV) has been developed by combining the fluorescence detector-based measurement of $X_{max}$ with the specific features of the surface detector signal through a parameter related to the air shower muon content, derived from the universality of the air shower development. No evidence of a statistically significant signal due to photon primaries was found using data collected in about 12 years of operation. Thus, upper bounds to the integral photon flux have been set using a detailed calculation of the detector exposure, in combination with a data-driven background estimation. The derived 95% confidence level upper limits are 0.0403, 0.01113, 0.0035, 0.0023, and 0.0021 km$^{-2}$ sr$^{-1}$ yr$^{-1}$ above 1, 2, 3, 5, and 10 EeV, respectively, leading to the most stringent upper limits on the photon flux in the EeV range. Compared with past results, the upper limits were improved by about 40% for the lowest energy threshold and by a factor 3 above 3 EeV, where no candidates were found and the expected background is negligible. The presented limits can be used to probe the assumptions on chemical composition of ultra-high energy cosmic rays and allow for the constraint of the mass and lifetime phase space of super-heavy dark matter particles.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2406.06697.md
    + _build/html/tmp_2406.06697/./regions.png
    + _build/html/tmp_2406.06697/./lineRGB_c2.png
    + _build/html/tmp_2406.06697/./velfield.png
    + _build/html/tmp_2406.06697/./fig_spec_panels.png
    + _build/html/tmp_2406.06697/./linemapne.png
    + _build/html/tmp_2406.06697/./linemapTe.png
    + _build/html/tmp_2406.06697/./linemapZ.png
exported in  _build/html/2406.07427.md
    + _build/html/tmp_2406.07427/././figs/SFig2.png
    + _build/html/tmp_2406.07427/././figs/SFig5.png
    + _build/html/tmp_2406.07427/././figs/Fig3.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\Zsun}{Z_\odot}$
$\newcommand{\Mbh}{M_{\rm BH}}$
$\newcommand{\Lhost}{L_{\rm host}}$
$\newcommand{\Mhost}{M_{\rm host}}$
$\newcommand{\Lya}{Ly\alpha}$
$\newcommand{\Ha}{H\alpha}$
$\newcommand{\Hb}{H\beta}$
$\newcommand{\Hg}{H\gamma}$
$\newcommand{\Hd}{H\delta}$
$\newcommand{\Hi}{H {\sc i}}$
$\newcommand{\Hii}{H {\sc ii}}$
$\newcommand{\Hei}{He {\sc i}}$
$\newcommand{\Heii}{He {\sc ii}}$
$\newcommand{\Ci}{[C {\sc i}]}$
$\newcommand{\Cii}{[C {\sc ii}]}$
$\newcommand{\Civ}{C {\sc iv}}$
$\newcommand{\Nii}{[N {\sc ii}]}$
$\newcommand{\Niv}{[N {\sc iv}]}$
$\newcommand{\Oi}{[O {\sc i}]}$
$\newcommand{\Oii}{[O {\sc ii}]}$
$\newcommand{\Oiii}{[O {\sc iii}]}$
$\newcommand{\Sii}{[S {\sc ii}]}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\um}{\mum}$
$\newcommand{\jykms}{Jy~km s^{-1}}$
$\newcommand{\Kkmspc}{K~km s^{-1} pc^2}$
$\newcommand{\lsim}{\mathrel{\rlap{\lower 3pt \hbox{\sim}} \raise 2.0pt \hbox{<}}}$
$\newcommand{\gsim}{\mathrel{\rlap{\lower 3pt \hbox{\sim}} \raise 2.0pt \hbox{>}}}$</div>



<div id="title">

# A quasar-galaxy merger at $z\sim 6.2$: rapid host growth via accretion of two massive satellite galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.06697-b31b1b.svg)](https://arxiv.org/abs/2406.06697)<mark>Appeared on: 2024-06-12</mark> -  _15 pages, 16 figures. Accepted for publication in A&A_

</div>
<div id="authors">

R. Decarli, et al. -- incl., <mark>F. Walter</mark>, <mark>E. Bañados</mark>, <mark>S. Bosman</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** We present JWST/NIRSpec Integral Field Spectroscopy in the rest-frame optical bands of the system PJ308--21, a quasar at $z=6.2342$ caught as its host galaxy interacts with companion galaxies. We detect spatially extended emission of several emission lines ( $\Ha$ , $\Hb$ , $\Oiii$ , $\Nii$ , $\Sii$ , $\Heii$ ), which we use to study the properties of the ionized phase of the interstellar medium: the source and hardness of the photoionizing radiation field, metallicity, dust reddening, electron density and temperature, and star formation. We also marginally detect continuum starlight emission associated with the companion sources. We find that at least two independent satellite galaxies are part of the system. While the quasar host appears highly enriched and obscured, with AGN-like photoionization conditions, the western companion shows minimal dust extinction, low metallicity ( $Z\sim0.4$ $\Zsun$ ), and star-formation driven photoionization. The eastern companion shows higher extinction and metallicity ( $Z\sim0.8$ $\Zsun$ ) compared to the western companion, and it is at least partially photoionized by the nearby quasar. We do not find any indication of AGN in the companion sources. Our study shows that while the quasar host galaxy is already very massive ( $M_{\rm dyn}>10^{11}$ $\Msun$ ), it is still rapidly building up by accreting two relatively massive ( $M_{\rm star}\sim 10^{10}$ $\Msun$ ) companion sources. This dataset showcases the power of JWST in exposing the build-up of massive galaxies in the first Gyr of the Universe.

</div>

<div id="div_fig1">

<img src="tmp_2406.06697/./regions.png" alt="Fig15.1" width="50%"/><img src="tmp_2406.06697/./lineRGB_c2.png" alt="Fig15.2" width="50%"/>

**Figure 15. -** * Left:* The $\Oi$ii$_{\rm 5007 Å}$ line emission map in the quasar+companion galaxies system PJ308--21 at $z$=6.2342, shown in gray scale (after PSF subtraction). The footprints of the NIRSpec IFU pointings are displayed in grey. We also mark the regions of the main components of the system with colored shading. * Right:* The same $\Oi$ii$_{\rm 5007 Å }$ map (in blue), now with the $\Ci$i$_{\rm 158 \mu m}$ map from \citet{decarli19} superimposed. The main regions are identified with ellipses: From left to right, the eastern companion (red), the outflow (orange), the quasar host galaxy (green), the bridge (cyan), and the western companion (blue). The $\Oi$ii  emission arises from the regions where $\Ci$i  is brightest, except for the "outflow" component where the $\Oi$ii  emission stretches beyond the $\Ci$i  emission. (*fig_system_map*)

</div>
<div id="div_fig2">

<img src="tmp_2406.06697/./velfield.png" alt="Fig2.1" width="50%"/><img src="tmp_2406.06697/./fig_spec_panels.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Velocity maps of $\Oi$ii$_{5007 Å}$ in the PJ308--21 system (* top panel*). The system shows a prominent velocity gradient: The western companion and the bridge connecting it to the quasar host galaxies show a blue shift of $\Delta v\approx-650$  and $-880$\kms  respectively compared to the system rest frame ($z=6.2342$, based on the $\Ci$i  emission). The host galaxy shows a prominent velocity gradient from South ($\Delta v\approx -700$\kms) to North ($\Delta v = +140$\kms). The eastern companion is redshifted with respect to the system's rest frame, with $\Delta v$ increasing at increasing distance from the quasar (up to $\Delta v \approx 650$\kms). The eastern companion shows a modest velocity dispersion ($\sigma_{\rm line}<140$\kms).  (*fig_gas_kin*)

</div>
<div id="div_fig3">

<img src="tmp_2406.06697/./linemapne.png" alt="Fig9.1" width="33%"/><img src="tmp_2406.06697/./linemapTe.png" alt="Fig9.2" width="33%"/><img src="tmp_2406.06697/./linemapZ.png" alt="Fig9.3" width="33%"/>

**Figure 9. -** Maps of the electron density, $n_e$({\rm top}), based on the $\Sii$$_{6717 Å }$/$\Sii$$_{6731 Å }$ ratio; of the electron temperature, $T_e$, based on the $\Oi$ii$_{5007 Å }$/$\Oi$ii$_{4363 Å }$ line ratio; and of the gas--phase metallicity $Z$, based on the Scal-PG16 method (see text for details). (*fig_map_electron_nT*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.06697"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\baselinestretch}{1.0}$
$\newcommand{\refname}{References in the main text}$
$\newcommand{\refname}{References in Methods}$
$\newcommand{\figurename}{Supplementary Figure}$
$\newcommand{\refname}{References in Supplementary information}$</div>



<div id="title">

# $\bf$ Support for fragile porous dust in a gravitationally self-regulated disk around IM Lup

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2406.07427-b31b1b.svg)](https://arxiv.org/abs/2406.07427)<mark>Appeared on: 2024-06-12</mark> -  _Accepted for publication. Author version_

</div>
<div id="authors">

<mark>T. Ueda</mark>, et al. -- incl., <mark>M. Flock</mark>, <mark>P. Sudarshan</mark>

</div>
<div id="abstract">

**Abstract:** Protoplanetary disks, the birthplace of planets, are expected to be gravitationally unstable in their early phase of evolution.IM Lup, a well-known T-Tauri star, is surrounded by a protoplanetary disk with spiral arms likely caused by gravitational instability.The IM Lup disk has been observed using various methods, but developing a unified explanatory model is challenging.Here we present a physical model of the IM Lup disk that offers a comprehensive explanation for diverse observations spanning from near-infrared to millimeter wavelengths.Our findings underscore the importance of dust fragility in retaining the observed millimeter emission and reveal the preference for moderately porous dust to explain observed millimeter polarization.We also find that the inner disk region is likely heated by gas accretion, providing a natural explanation for bright millimeter emission within 20 au.The actively heated inner region in the model casts a 100-au-scale shadow, aligning seamlessly with the near-infrared scattered light observation.The presence of accretion heating also supports the fragile dust scenario in which accretion efficiently heat the disk midplane.Due to the fragility of dust, it is unlikely that a potential embedded planet at 100 au formed via pebble accretion in a smooth disk, pointing to local dust enhancement boosting pebble accretion or alternative pathways such as outward migration or gravitational fragmentation.

</div>

<div id="div_fig1">

<img src="tmp_2406.07427/././figs/SFig2.png" alt="Fig8" width="100%"/>

**Figure 8. -** 
Opacities at $\lambda=1.3$ mm obtained from our dust model.
Left: absorption opacity.
Right: polarization efficiency which is described by the product of the degree of linear polarization at scattering angle of $90^{\circ}$ and effective albedo.
The red solid line and blue dashed line denote compact ($f=0$) and porous ($f=0.2$) dust model, respectively.
The green dash-dotted line denotes the dust opacity with $f=0.01$ for reference.
The dust size distribution is assumed to have a power-law index of $-3$.
 (*fig:opac*)

</div>
<div id="div_fig2">

<img src="tmp_2406.07427/././figs/SFig5.png" alt="Fig11" width="100%"/>

**Figure 11. -** 
Summary of our model of the gas disk around IM Lup.
The black solid line denotes our model.
The blue solid line and red dotted line denotes the model with temperature solely determined by stellar irradiation ($T_{\rm MAPS}$; Eq. \ref{eq:Sierra}) and by accretion heating ($T_{\rm GSR,acc}$; Eq. \ref{eq:T_acc}), respectively.
The blue dashed lines show the conventional model \cite{Zhang+21} for which we assume $h_{\rm g}=c_{\rm s}/\Omega_{\rm K}$.
Left: midplane temperature profile. The gray solid line in the left panel denotes the observed brightness temperature at $\lambda=1.25 {\rm mm}$ taken by the ALMA DSHARP program \cite{Andrews+18}.
The green dash-dotted line denotes the temperature determined by external radiation (15 K).
Middle: gas surface density. The green dash-dotted line denotes the Minimum-Mass Solar Nebula model \cite{Hayashi81}.
Left: accretion parameter $\alpha_{\rm acc}$.
 (*fig:disk*)

</div>
<div id="div_fig3">

<img src="tmp_2406.07427/././figs/Fig3.png" alt="Fig1" width="100%"/>

**Figure 1. -** 
Comparison of the 1.25 mm intensity profile obtained from our models and the DSHARP observation.
The red, green, blue and purple solid lines denote the model with $v_{\rm f}=0.3$, 1, 3 and 10 ${\rm m s^{-1}}$, respectively, while the black solid line denotes the observed value obtained from DSHARP observation \cite{Andrews+18}.
The left panel shows our standard model in which the disk temperature is determined by accretion heating and external irradiation, while the right panel shows the model based on the temperature model in which the heating is solely by stellar irradiation (Eq. \ref{eq:Sierra}).
 (*fig:dsharp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2406.07427"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

92  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
